In [1]:
import pandas as pd
import numpy as np
from quetzal.model import stepmodel

In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none func

# Calibration
## Preparation of input data
Synthesise travel time columns with optimal paths from the model

In [2]:
input_path = '../input/transport_demand/'
model_path = '../model/'

In [3]:
# Load StepModel with LOS table...
sm = stepmodel.read_json(model_path + 'de_pt_access_egress')
pt = stepmodel.read_json(model_path + 'de_pt_los')
car = stepmodel.read_json(model_path + 'de_road_los')

In [4]:
sm.pt_los = pt.pt_los
sm.car_los = car.car_los

In [5]:
# Load calibration dataset
trips = pd.read_csv(input_path + 'calibration_inter-cellular_trips_MiD2017.csv')
print(len(trips.index))

101172


In [6]:
trips = trips.drop(columns='Unnamed: 0')
trips.reset_index(drop=True, inplace=True)
#trips.head()

In [7]:
# Replace LAU codes with NUTS IDs
if not str(trips.loc[0, 'origin']).startswith('DE'):
    z = stepmodel.read_json(model_path + 'de_zones')
    lau_nuts_dict = z.zones.set_index('lau_id')['NUTS_ID'].to_dict()
    trips['origin'] = trips['origin'].astype(str).map(lau_nuts_dict)
    trips['destination'] = trips['destination'].astype(str).map(lau_nuts_dict)

    # Drop all other trips
    trips = trips.loc[(trips['origin'].notna()) & (trips['destination'].notna())]
    len(trips.index)

All imputated time columns are needed for the MLE model. They do not replace duration statements in the survey but complement each trip with alternative travel times in modes that were not used for this trip.

In [8]:
# Create dictionaries from LoS tables
# Take the fastest trips
car_dict = sm.car_los.sort_values('time').groupby(
    ['origin', 'destination']).first()['time'].to_dict()
pt_dict = sm.pt_los.sort_values('time').groupby(
    ['origin', 'destination', 'route_type']).first()['time'].to_dict()

In [9]:
# Car
# Directly from car LoS table, converted to minutes
trips['time_car'] = [car_dict[a]/60
                     if a in car_dict.keys() else np.nan
                     for a in tuple(zip(trips['origin'], trips['destination']))]

In [11]:
# Some OD pairs might not be in the volumes table
# which results in missing values for the calibration dataset
k = len(trips.loc[trips['time_car'].isna()])
print(k)
print(k / len(trips.loc[trips['mode_model']==6]))

35
0.00039677591230118694


In [12]:
# Drop trips without time reference if its share is insignificant
trips = trips.loc[trips['time_car'].notna()]

In [13]:
# PT
for t in sm.pt_los['route_type'].unique():
    trips['route_type'] = t
    trips['time_' + t] = [pt_dict[a]/60
        if a in pt_dict.keys() else np.inf
        for a in tuple(zip(trips['origin'], trips['destination'], trips['route_type']))]
trips.drop('route_type', axis=1, inplace=True)

In [14]:
# Replace time for the mode of choice
mode_col_map = {1: 'time_rail_short_distance', 2: 'time_rail_long_distance',
                3: 'time_coach', 4: 'time_bus', 5: 'time_air',
                6: 'time_car', 7: 'time_walk'}
for index, row in trips.iterrows():
    trips.loc[index, mode_col_map[row['mode_model']]] = row['duration']

In [15]:
# The fastest route determines time of rail service
trips['time_rail'] = [min(t_s, t_l)
                      for t_s, t_l in zip(trips['time_rail_short_distance'],
                                          trips['time_rail_long_distance'])]
trips.loc[trips['mode_model']==1, 'time_rail'] = \
    trips.loc[trips['mode_model']==1, 'time_rail_short_distance']
trips.loc[trips['mode_model']==2, 'time_rail'] = \
    trips.loc[trips['mode_model']==2, 'time_rail_long_distance']

In [16]:
trips.groupby('mode_model').agg(
    {'time_rail_long_distance': 'mean', 'time_rail_short_distance': 'mean',
     'time_rail': 'mean', 'time_coach': 'mean', 'time_bus': 'mean',
     'time_air': 'mean', 'time_car': 'mean', 'time_walk': 'mean'})

,time_rail_long_distance,time_rail_short_distance,time_rail,time_coach,time_bus,time_air,time_car,time_walk
mode_model,,,,,,,,
1,inf,65.846534,65.846534,inf,inf,inf,52.163362,inf
2,167.203851,inf,167.203851,inf,inf,inf,160.464892,inf
3,inf,inf,inf,104.176471,inf,inf,100.487535,inf
4,inf,inf,inf,inf,46.159036,inf,62.338265,inf
5,inf,inf,inf,inf,inf,122.5,332.250000,inf
6,inf,inf,inf,inf,inf,inf,40.676733,inf
7,inf,inf,inf,inf,inf,inf,80.816343,54.32687


In [17]:
trips.replace({np.inf:np.nan}).groupby('mode_model').agg(
    {'time_rail_long_distance': 'mean', 'time_rail_short_distance': 'mean',
     'time_rail': 'mean', 'time_coach': 'mean', 'time_bus': 'mean',
     'time_air': 'mean', 'time_car': 'mean', 'time_walk': 'mean'})

,time_rail_long_distance,time_rail_short_distance,time_rail,time_coach,time_bus,time_air,time_car,time_walk
mode_model,,,,,,,,
1,135.518142,65.846534,65.846534,130.443089,91.870660,265.359350,52.163362,55.793799
2,167.203851,174.541633,167.203851,235.366255,103.670423,242.954056,160.464892,60.218333
3,196.069198,138.887360,136.070088,104.176471,89.501361,296.813194,100.487535,52.270833
4,197.714559,109.995305,110.781983,198.911050,46.159036,264.042808,62.338265,53.486597
5,291.546032,420.285417,287.009420,415.366667,NaN,122.500000,332.250000,71.433333
6,175.157235,105.244177,104.620583,172.707642,90.373312,291.408682,40.676733,56.678750
7,216.705586,130.428461,129.233836,236.993995,91.436594,283.895628,80.816343,54.326870


In [18]:
# How many observations must be dropped later and what are their shares?
mode_dict = {1: 'rail_short_distance', 2: 'rail_long_distance',
             3: 'coach', 4: 'bus', 5: 'air', 6: 'car', 7: 'walk'}
mode_dict = {v:k for k,v in mode_dict.items()}
for t in mode_dict.keys():
    k = len(trips.loc[((trips['mode_model']==mode_dict[t]) & (trips['time_'+t]==np.inf))])
    print('{}: {} observations have no model connection ({} percent)'.format(
        t, k, k/len(trips.loc[trips['mode_model']==mode_dict[t]])*100))

rail_short_distance: 0 observations have no model connection (0.0 percent)
rail_long_distance: 0 observations have no model connection (0.0 percent)
coach: 0 observations have no model connection (0.0 percent)
bus: 0 observations have no model connection (0.0 percent)
air: 0 observations have no model connection (0.0 percent)
car: 0 observations have no model connection (0.0 percent)
walk: 0 observations have no model connection (0.0 percent)


### Save calibration dataset

In [19]:
trips.to_csv(input_path + 'calibration_inter-cellular_trips_MiD2017.csv')